In [1]:
# users table 에 패스워드 넣기
import json
import pymysql

In [2]:
with open('mysql.json', 'r') as file:
    config_str = file.read()
config_str
config = json.loads(config_str)

In [3]:
conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database = config['database'],
    port = config['port']
)

In [4]:
# 패스워드 암호화
import hashlib
import base64

In [5]:
# secure hash algorithm(SHA-256)
pwd = '1234'
pwd_sha256 = hashlib.sha256(pwd.encode())
pwd_sha256.digest()

b'\x03\xacgB\x16\xf3\xe1\\v\x1e\xe1\xa5\xe2U\xf0g\x956#\xc8\xb3\x88\xb4E\x9e\x13\xf9x\xd7\xc8F\xf4'

In [6]:
# base64 인코딩 
base64.b64encode(pwd_sha256.digest())

b'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [9]:
hashed_pwd =base64.b64encode(pwd_sha256.digest()).decode('utf-8')
hashed_pwd

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [10]:
len(hashed_pwd)

44

In [11]:
# 패스워드를 생성하는 함수
def gen_pwd(pwd):
    hashed_pwd =base64.b64encode(pwd_sha256.digest()).decode('utf-8')
    return hashed_pwd

In [12]:
# users table을 읽어서 pwd 열에 패스워드를 채움
sql = 'SELECT uid FROM users;'
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall()
results

(('admin',), ('djy',), ('eskim',), ('wjlee',))

In [13]:
sql = 'UPDATE users SET pwd=%s where uid=%s;'
cur = conn.cursor()
for row in results:
    uid = row[0]
    pwd = gen_pwd(uid+'12')
    cur.execute(sql, (pwd, uid))
conn.commit()

In [14]:
# 로그인 
# 1)uid OK, pwd OK 
uid, pwd = 'admin', 'admin12'
sql = "SELECT uid, pwd FROM users WHERE uid=%s;"
cur = conn.cursor()
cur.execute(sql, (uid,))
result = cur.fetchone()
db_pwd = result[1]
db_pwd

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [15]:
hashed_pwd = gen_pwd(pwd)
hashed_pwd

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [16]:
# uid OK, pwd BAD
pwd = '1234'
hashed_pwd = gen_pwd(pwd)
hashed_pwd

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [17]:
# 3 uid BAD
uid = 'fool'
sql = "SELECT uid, pwd FROM users WHERE uid=%s;"
cur = conn.cursor()
cur.execute(sql, (uid,))



0

In [18]:
result = cur.fetchone()

In [19]:
result

In [21]:
uid, pwd = 'admin12', 'admin1234'

In [22]:
sql = "SELECT uid, pwd FROM users WHERE uid=%s and is_deleted=0;"
cur = conn.cursor()
cur.execute(sql, (uid,))
result = cur.fetchone()
if result:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('correct')
    else:
        print('incorrect password')
else:
    print('bad uid')

bad uid


In [23]:
cur.close()
conn.close()


In [24]:
x = []
if x:
    print('true')
else:
    print('false')

false
